In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from PIL import Image
from tqdm import tqdm
from pandas_profiling import ProfileReport

import keras, re, string
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [31]:
df = pd.read_csv("../data.csv")

In [32]:
df.columns

Index(['Unnamed: 0', 'index', 'totPurchaseAmt', 'street1', 'zip', 'property',
       'ZillowURL', 'zpid', 'streetAddress', 'zipcode', 'city', 'state',
       'latitude', 'longitude', 'dateSold', 'bathrooms', 'bedrooms',
       'livingArea', 'yearBuilt', 'lotSize', 'homeType', 'homeStatus',
       'photoCount', 'imageLink', 'daysOnZillow', 'isFeatured',
       'shouldHighlight', 'brokerId', 'zestimate', 'rentZestimate',
       'listing_sub_type', 'priceReduction', 'isUnmappable', 'rentalPetsFlags',
       'mediumImageLink', 'isPreforeclosureAuction', 'homeStatusForHDP',
       'festimate', 'isListingOwnedByCurrentSignedInAgent',
       'isListingClaimedByCurrentSignedInUser', 'hiResImageLink',
       'watchImageLink', 'tvImageLink', 'tvCollectionImageLink',
       'tvHighResImageLink', 'zillowHasRightsToImages',
       'desktopWebHdpImageLink', 'isNonOwnerOccupied', 'hideZestimate',
       'isPremierBuilder', 'isZillowOwned', 'currency', 'country',
       'taxAssessedValue', 'streetAddr

In [33]:
df = df[df["totPurchaseAmt"] >= 50000]
df = df[df["totPurchaseAmt"] <= 5000000]
df.drop_duplicates("mediumImageLink", keep = "first", inplace = True)

df = df[
    ["totPurchaseAmt", "latitude", "longitude", "bathrooms", 
     "bedrooms", "livingArea", "yearBuilt", "lotSize",
     "averageSchoolRating", "zip", "description"]
]

# df = pd.get_dummies(data=df, columns=["bathrooms", "bedrooms", "yearBuilt", "zip"])
df.dropna(inplace=True)

In [ ]:
profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [25]:
def remove_mypunct(corpus):

    corpus = map(lambda x: re.sub(r"(#|@|http)\S+", "", x), corpus) #get rid of hashtags 
    corpus = map(lambda x: re.sub(r"(\t|\n|\v|…|“|”)", "", x), corpus) #get rid of whitesape
    corpus = map(lambda x:  re.sub(r"""\w*\d\w*""", ' ', x.lower()), corpus) #get rid of numbers
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    corpus = map(lambda x: punc_re.sub(' ', x), corpus)
    corpus = map(lambda x: re.sub('[\W_]+', " ", x), corpus) #get rid of emojis
    
    return list(corpus)

sid = SentimentIntensityAnalyzer()
score_tuples = []
score = []

for i in df.index.values:
    clean_house = remove_mypunct([df.description[i]])
    ss = sid.polarity_scores(clean_house[0])
    score_tuples.append((float(ss['compound']), df.description[i], i))
    score.append(float(ss['compound']))
    df.at[i, 'sentiment'] = float(ss['compound'])

# df["description_sentiment"] = 

In [27]:
df.head()

,totPurchaseAmt,latitude,longitude,livingArea,lotSize,averageSchoolRating,description,bathrooms_0.5,bathrooms_0.75,bathrooms_1.0,bathrooms_1.2,bathrooms_1.25,bathrooms_1.5,bathrooms_1.75,bathrooms_2.0,bathrooms_2.25,bathrooms_2.5,bathrooms_2.75,bathrooms_3.0,bathrooms_3.25,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_4.75,bathrooms_5.0,bathrooms_5.5,bathrooms_6.0,bathrooms_7.0,bathrooms_8.0,bathrooms_9.0,bathrooms_13.0,bedrooms_0.0,bedrooms_1.0,bedrooms_2.0,bedrooms_3.0,bedrooms_4.0,bedrooms_5.0,bedrooms_6.0,bedrooms_7.0,bedrooms_8.0,bedrooms_9.0,yearBuilt_-1,yearBuilt_1850,yearBuilt_1854,yearBuilt_1858,yearBuilt_1859,yearBuilt_1864,yearBuilt_1865,yearBuilt_1868,yearBuilt_1870,yearBuilt_1871,yearBuilt_1872,yearBuilt_1874,yearBuilt_1875,yearBuilt_1879,yearBuilt_1880,yearBuilt_1881,yearBuilt_1882,yearBuilt_1883,yearBuilt_1884,yearBuilt_1885,yearBuilt_1886,yearBuilt_1887,yearBuilt_1888,yearBuilt_1889,yearBuilt_1890,yearBuilt_1891,yearBuilt_1892,yearBuilt_1893,yearBuilt_1894,yearBuilt_1895,yearBuilt_1896,yearBuilt_1897,yearBuilt_1898,yearBuilt_1899,yearBuilt_1900,yearBuilt_1901,yearBuilt_1902,yearBuilt_1903,yearBuilt_1904,yearBuilt_1905,yearBuilt_1906,yearBuilt_1907,yearBuilt_1908,yearBuilt_1909,yearBuilt_1910,yearBuilt_1911,yearBuilt_1912,yearBuilt_1913,yearBuilt_1914,yearBuilt_1915,yearBuilt_1916,yearBuilt_1917,yearBuilt_1918,yearBuilt_1919,yearBuilt_1920,yearBuilt_1921,yearBuilt_1922,yearBuilt_1923,yearBuilt_1924,yearBuilt_1925,yearBuilt_1926,yearBuilt_1927,yearBuilt_1928,yearBuilt_1929,yearBuilt_1930,yearBuilt_1931,yearBuilt_1932,yearBuilt_1933,yearBuilt_1934,yearBuilt_1935,yearBuilt_1936,yearBuilt_1937,yearBuilt_1938,yearBuilt_1939,yearBuilt_1940,yearBuilt_1941,yearBuilt_1942,yearBuilt_1943,yearBuilt_1944,yearBuilt_1945,yearBuilt_1946,yearBuilt_1947,yearBuilt_1948,yearBuilt_1949,yearBuilt_1950,yearBuilt_1951,yearBuilt_1952,yearBuilt_1953,yearBuilt_1954,yearBuilt_1955,yearBuilt_1956,yearBuilt_1957,yearBuilt_1958,yearBuilt_1959,yearBuilt_1960,yearBuilt_1961,yearBuilt_1962,yearBuilt_1963,yearBuilt_1964,yearBuilt_1965,yearBuilt_1966,yearBuilt_1967,yearBuilt_1968,yearBuilt_1969,yearBuilt_1970,yearBuilt_1971,yearBuilt_1972,yearBuilt_1973,yearBuilt_1974,yearBuilt_1975,yearBuilt_1976,yearBuilt_1977,yearBuilt_1978,yearBuilt_1979,yearBuilt_1980,yearBuilt_1981,yearBuilt_1982,yearBuilt_1983,yearBuilt_1984,yearBuilt_1985,yearBuilt_1986,yearBuilt_1987,yearBuilt_1988,yearBuilt_1989,yearBuilt_1990,yearBuilt_1991,yearBuilt_1992,yearBuilt_1993,yearBuilt_1994,yearBuilt_1995,yearBuilt_1996,yearBuilt_1997,yearBuilt_1998,yearBuilt_1999,yearBuilt_2000,yearBuilt_2001,yearBuilt_2002,yearBuilt_2003,yearBuilt_2004,yearBuilt_2005,yearBuilt_2006,yearBuilt_2007,yearBuilt_2008,yearBuilt_2009,yearBuilt_2010,yearBuilt_2011,yearBuilt_2012,yearBuilt_2013,yearBuilt_2014,yearBuilt_2015,yearBuilt_2016,yearBuilt_2017,yearBuilt_2018,yearBuilt_2019,yearBuilt_2020,zip_10050.0,zip_53590.0,zip_54020.0,zip_54433.0,zip_55001.0,zip_55003.0,zip_55005.0,zip_55009.0,zip_55011.0,zip_55014.0,zip_55016.0,zip_55018.0,zip_55024.0,zip_55025.0,zip_55031.0,zip_55033.0,zip_55034.0,zip_55038.0,zip_55040.0,zip_55042.0,zip_55043.0,zip_55044.0,zip_55054.0,zip_55055.0,zip_55056.0,zip_55057.0,zip_55065.0,zip_55068.0,zip_55070.0,zip_55071.0,zip_55073.0,zip_55075.0,zip_55076.0,zip_55077.0,zip_55079.0,zip_55082.0,zip_55085.0,zip_55089.0,zip_55092.0,zip_55101.0,zip_55102.0,zip_55103.0,zip_55104.0,zip_55105.0,zip_55106.0,zip_55107.0,zip_55108.0,zip_55109.0,zip_55110.0,zip_55112.0,zip_55113.0,zip_55114.0,zip_55115.0,zip_55116.0,zip_55117.0,zip_55118.0,zip_55119.0,zip_55120.0,zip_55121.0,zip_55122.0,zip_55123.0,zip_55124.0,zip_55125.0,zip_55126.0,zip_55127.0,zip_55128.0,zip_55129.0,zip_55130.0,zip_55132.0,zip_55243.0,zip_55303.0,zip_55304.0,zip_55305.0,zip_55306.0,zip_55308.0,zip_55311.0,zip_55315.0,zip_55316.0,zip_55317.0,zip_55318.0,zip_55320.0,zip_55322.0,zip_55327.0,zip_55328.0,zip_55330.0,zip_55331.0,zip_55337.0,zip_55338.0,zip_55339.0,zip_55340.0,zip_55341.0,zip_55343.0,zip_55344.0,z

In [4]:
X = df.drop("totPurchaseAmt", axis=1)
y = df["totPurchaseAmt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]
#add model layers
model.add(Dense(128, activation='relu', input_shape=(n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=["mae"])

In [7]:
model.fit(X_train, y_train, validation_split=0.1, epochs=500)


Train on 10461 samples, validate on 1163 samples
Epoch 1/500
10461/10461 [==============================] - 1s 77us/step - loss: 324400.7066 - mae: 324400.7188 - val_loss: 357101.3929 - val_mae: 357101.3750

KeyboardInterrupt: 